In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
# from matplotlib.finance import candlestick
from matplotlib.dates import DateFormatter, WeekdayLocator, DayLocator, MONDAY, date2num
import seaborn as sns
import time
import random
from sklearn import linear_model
from statsmodels.tsa.stattools import adfuller as adf
import itertools
import scipy
from scipy.stats import pearsonr
from statsmodels.tsa.stattools import coint  
import statsmodels.api as sm
import statsmodels.tsa.stattools as tsa


data = pd.read_csv('data.csv')


In [12]:
data.columns.tolist()
data.rename(columns={'\ufeffDate': 'date'}, inplace=True)

data['date'] = data['date'].apply(lambda d: datetime.datetime.strptime(d, "%Y-%m-%d"))
data=data.set_index(['date'])

In [ ]:
Banks=data[['BAC US Equity','BBT US Equity','C US Equity','JPM US Equity','KEY US Equity','MTB US Equity',
            'PNC US Equity','STI US Equity','USB US Equity','WFC US Equity']]

Electricity= data[['AEP US Equity','D US Equity','ED US Equity','EXC US Equity','NEE US Equity','PCG US Equity',
                   'PEG US Equity','PPL US Equity','SO US Equity','XEL US Equity']]

Financial_Sercives= data[['AXP US Equity','BK US Equity','BLK US Equity','CME US Equity','COF US Equity',
                          'GS US Equity','MA US Equity','MS US Equity','SCHW US Equity','V US Equity']]

Gas_Water_MultiUlities = data[['AEE US Equity','AWK US Equity','CNP US Equity','DUK US Equity','NI US Equity',
                               'OKE US Equity','SRE US Equity','WEC US Equity']]

Life_Insurance= data[['AFL US Equity','AIZ US Equity','BHF US Equity','LNC US Equity','MET US Equity','PFG US Equity',
                      'PRU US Equity','TMK US Equity','UNM US Equity']]


Non_Life_Insurance= data[['AIG US Equity','ALL US Equity','AON US Equity','BRK/A US Equity','CB US Equity',
                          'MMC US Equity','PGR US Equity','TRV US Equity','WLTW US Equity']]

Oil_Gas_Producers= data[['APC US Equity','COP US Equity','CVX US Equity','EOG US Equity','MPC US Equity',
                         'OXY US Equity','PSX US Equity','PXD US Equity','VLO US Equity','XOM US Equity']]


Oil_Equipment_Services_Distribution=data[['BHGE US Equity','FTI US Equity','HAL US Equity','HP US Equity',
                                          'KMI US Equity','NOV US Equity','SLB US Equity','WMB US Equity']]

Real_Estate_Investment_Trusts=data[['AMT US Equity','AVB US Equity','CCI US Equity','DLR US Equity','EQIX US Equity',
                                    'PLD US Equity','PSA US Equity','SPG US Equity','WY US Equity']]

Software_Computer_Services =data[['ADBE US Equity','CRM US Equity','CTSH US Equity','FB US Equity','GOOGL US Equity',
                                  'IBM US Equity','INTU US Equity','MSFT US Equity','ORCL US Equity']]

Technology_Hardware_Equipment=data[['AAPL US Equity','AMAT US Equity','AVGO US Equity','CSCO US Equity',
                                    'INTC US Equity','NVDA US Equity','QCOM US Equity','TXN US Equity']]


In [14]:
Data_new= [Banks,Electricity,Financial_Sercives,Gas_Water_MultiUlities,Life_Insurance,
           Non_Life_Insurance,Oil_Gas_Producers,Oil_Equipment_Services_Distribution,Real_Estate_Investment_Trusts,
           Software_Computer_Services,Technology_Hardware_Equipment]

In [53]:

def pair_identification(data, verbose=True):

    cointegrated_pairs = []
    pearson_coeff = []
    p_value = []
    test_value = []
    gamma_coeff = []
    res_sd = []    
    counter = 0
    start_date_dt = data.index[0].date().strftime("%Y-%m") 
    end_date_dt = data.index[-1].date().strftime("%Y-%m")
    ticket_list = data.columns.tolist()
    for stock_pair in itertools.combinations(ticket_list, 2):
        X = data.ix[start_date_dt:end_date_dt, stock_pair[0]]
        Y = data.ix[start_date_dt:end_date_dt, stock_pair[1]]
        corr_coeff = pearsonr(X, Y)[0]
        if corr_coeff >= 0.90:
            if coint(X, Y)[1] < 0.1:
                reg = sm.OLS(X, Y).fit()
                res = reg.resid
                ADF_test = tsa.adfuller(res)
                if ADF_test[1] < 0.1:
                    cointegrated_pairs.append(stock_pair)
                    p_value.append(ADF_test[1])
                    test_value.append(ADF_test[0])
                    gamma_coeff.append(reg.params[0])
                    pearson_coeff.append(corr_coeff)
                    res_sd.append(res.std())    
                    counter += 1
    if counter != 0:
        index = pd.MultiIndex.from_tuples(cointegrated_pairs, names=['Stock_A', 'Stock_B'])
        df = pd.DataFrame(data={'pearson':pearson_coeff, 'ADF_test_value':test_value, 'p_value':p_value,
                                'gamma':gamma_coeff, 'sd_residuals':res_sd}, index=index)
        return df

In [54]:
for f in Data_new:
     print(pair_identification(f, verbose=True))

None
None
                             ADF_test_value     gamma   p_value   pearson  \
Stock_A       Stock_B                                                       
AXP US Equity CME US Equity       -4.350499  0.640905  0.000363  0.968856   
BLK US Equity V US Equity         -3.354510  4.481817  0.012611  0.950739   

                             sd_residuals  
Stock_A       Stock_B                      
AXP US Equity CME US Equity      3.257995  
BLK US Equity V US Equity       17.924448  
None
None
                              ADF_test_value     gamma   p_value   pearson  \
Stock_A       Stock_B                                                        
AON US Equity WLTW US Equity       -4.324095  0.891190  0.000403  0.964122   
CB US Equity  MMC US Equity        -2.776518  1.867407  0.061707  0.978368   
MMC US Equity WLTW US Equity       -4.088751  0.536116  0.001011  0.934508   

                              sd_residuals  
Stock_A       Stock_B                       
AON US Equity 